In [1]:
import os, json, re, stat, pathlib, base64
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from pathlib import Path

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
def jaccard_similarity(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return float(intersection/union) if union !=0 else 0

In [59]:
DATA_PATH = "/Users/ethancoskay/projects/MovieRecs/data/TMDB_all_movies.csv"
df_main = pd.read_csv(DATA_PATH)
df_main.head()
df_main.tail()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,original_language,original_title,overview,popularity,tagline,genres,production_companies,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path
1088901,1473136,Will You Be My Cancer?,0.0,0.0,Released,NaN,0.0,0.0,0.0,NaN,en,Will You Be My Cancer?,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1088902,1473137,Secret Bee,0.0,0.0,Released,NaN,0.0,2.0,0.0,NaN,en,Secret Bee,Secret Bee ventures toward the sweet honey- sp...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1088903,1473138,Sacrebleu,0.0,0.0,Released,NaN,0.0,0.0,0.0,tt2512636,en,Sacrebleu,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.4,12.0,NaN
1088904,2662126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Johannes Allmayer, Jules Waringo, Vicky Krieps...",Florian Gallenberger,Kolja Brandt,"Florian Gallenberger, Torsten Wenzel, Tom Kinn...","Nicolas Steil, Benjamin Herrmann",NaN,NaN,NaN,NaN
1088905,5180730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Hailee Steinfeld, Tim Martin Gleason, Holland ...",Travis Knight,Enrique Chediak,"Christina Hodson, Shoji Kawamori","Chris Brigham, Stephen Davis, Don Murphy, Bria...",Dario Marianelli,NaN,NaN,NaN


In [56]:

df2 = df_main[['id', 'title', 'overview', 'genres', 'cast', 'director', 'imdb_rating', 'release_date']]
df = df_main[['id', 'title', 'overview', 'genres', 'cast', 'director', 'writers', 'imdb_rating']]
df2.to_pickle('all_movies.pkl')

In [26]:
cols_to_lower = ['director', 'writers', 'cast', 'genres']
for col in cols_to_lower:
    df[col] = df[col].fillna('').str.lower().str.replace(r"\s+", "", regex=True).str.split(",")
    df[col] = df[col].apply(lambda x: x[:4] if len(x)>4 else x)


df["overview"] = df["overview"].fillna('').str.lower()

/var/folders/39/8v7z_shs5qd71d5q4_6m776w0000gn/T/ipykernel_9895/3294047156.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna('').str.lower().str.replace(r"\s+", "", regex=True).str.split(",")
/var/folders/39/8v7z_shs5qd71d5q4_6m776w0000gn/T/ipykernel_9895/3294047156.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: x[:4] if len(x)>4 else x)
/var/folders/39/8v7z_shs5qd71d5q4_6m776w0000gn/T/ipykernel_9895/3294047156.py:7: SettingWithCopyWarning: 


In [27]:
df.head()

,id,title,overview,genres,cast,director,writers,imdb_rating
0,2,Ariel,a finnish man goes to the city to find a job a...,"[comedy, drama, romance, crime]","[merjapulkkinen, eetuhilkamo, turopajala, esko...",[akikaurismäki],[akikaurismäki],7.4
1,3,Shadows in Paradise,"nikander, a rubbish collector and would-be ent...","[comedy, drama, romance]","[eskonikkari, marirantasila, marinamartinoff, ...",[akikaurismäki],[akikaurismäki],7.4
2,5,Four Rooms,it's ted the bellhop's first night on the job....,"[comedy, crime]","[quinnhellerman, lawrencebender, timroth, lana...","[allisonanders, robertrodriguez, alexandrerock...","[allisonanders, robertrodriguez, alexandrerock...",6.7
3,6,Judgment Night,"four young friends, while taking a shortcut en...","[action, crime, thriller]","[michaelscranton, hankmcgill, donovand.ross, n...",[stephenhopkins],"[jerecunningham, lewiscolick]",6.6
4,8,Life in Loops (A Megacities RMX),timo novotny labels his new project an experim...,[documentary],[],[timonovotny],"[michaelglawogger, timonovotny]",8.2


In [28]:
df.cast[0]

['merjapulkkinen', 'eetuhilkamo', 'turopajala', 'eskosalminen']

In [29]:
cols_to_merge = ['director', 'writers', 'cast', 'genres']

# Function to join lists from multiple columns, convert to set (for uniqueness), then back to list
def merge_and_unique(row):
    combined = []
    for col in cols_to_merge:
        combined.extend(row[col])  # Add the list from each column to combined list
    return list(set(combined))  # Convert to set for uniqueness, then back to list

# Apply the function to each row
df['tags'] = df.apply(merge_and_unique, axis=1)

/var/folders/39/8v7z_shs5qd71d5q4_6m776w0000gn/T/ipykernel_9895/436728637.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df.apply(merge_and_unique, axis=1)


In [30]:
df.drop(columns=cols_to_merge, inplace=True)

/var/folders/39/8v7z_shs5qd71d5q4_6m776w0000gn/T/ipykernel_9895/708697862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=cols_to_merge, inplace=True)


In [31]:
df.set_index('id', inplace=True)

In [32]:
df.head()

,title,overview,imdb_rating,tags
id,,,,
2,Ariel,a finnish man goes to the city to find a job a...,7.4,"[comedy, merjapulkkinen, romance, eetuhilkamo,..."
3,Shadows in Paradise,"nikander, a rubbish collector and would-be ent...",7.4,"[jukka-pekkapalo, romance, marirantasila, esko..."
5,Four Rooms,it's ted the bellhop's first night on the job....,6.7,"[robertrodriguez, quinnhellerman, quentintaran..."
6,Judgment Night,"four young friends, while taking a shortcut en...",6.6,"[lewiscolick, thriller, donovand.ross, nigelgi..."
8,Life in Loops (A Megacities RMX),timo novotny labels his new project an experim...,8.2,"[, documentary, michaelglawogger, timonovotny]"


In [33]:
df['tags'] = df['tags'].apply(lambda x: [item for item in x if item != ''] if isinstance(x, list) else x)
df.head()

/var/folders/39/8v7z_shs5qd71d5q4_6m776w0000gn/T/ipykernel_9895/1518854655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: [item for item in x if item != ''] if isinstance(x, list) else x)


,title,overview,imdb_rating,tags
id,,,,
2,Ariel,a finnish man goes to the city to find a job a...,7.4,"[comedy, merjapulkkinen, romance, eetuhilkamo,..."
3,Shadows in Paradise,"nikander, a rubbish collector and would-be ent...",7.4,"[jukka-pekkapalo, romance, marirantasila, esko..."
5,Four Rooms,it's ted the bellhop's first night on the job....,6.7,"[robertrodriguez, quinnhellerman, quentintaran..."
6,Judgment Night,"four young friends, while taking a shortcut en...",6.6,"[lewiscolick, thriller, donovand.ross, nigelgi..."
8,Life in Loops (A Megacities RMX),timo novotny labels his new project an experim...,8.2,"[documentary, michaelglawogger, timonovotny]"


In [34]:
df["tags"][8]

['documentary', 'michaelglawogger', 'timonovotny']

In [35]:
# Count the number of duplicates
df['title'].duplicated().sum()

166942

In [36]:
df = df.drop_duplicates(subset=['title'], keep='first')

In [37]:
df[df['title'].duplicated()]

,title,overview,imdb_rating,tags
id,,,,


In [38]:
df.shape

(921964, 4)

In [39]:
df.head()

,title,overview,imdb_rating,tags
id,,,,
2,Ariel,a finnish man goes to the city to find a job a...,7.4,"[comedy, merjapulkkinen, romance, eetuhilkamo,..."
3,Shadows in Paradise,"nikander, a rubbish collector and would-be ent...",7.4,"[jukka-pekkapalo, romance, marirantasila, esko..."
5,Four Rooms,it's ted the bellhop's first night on the job....,6.7,"[robertrodriguez, quinnhellerman, quentintaran..."
6,Judgment Night,"four young friends, while taking a shortcut en...",6.6,"[lewiscolick, thriller, donovand.ross, nigelgi..."
8,Life in Loops (A Megacities RMX),timo novotny labels his new project an experim...,8.2,"[documentary, michaelglawogger, timonovotny]"


In [40]:
df.isnull().sum()

title               1
overview            0
imdb_rating    543892
tags                0
dtype: int64

In [41]:
df.dropna(inplace=True)

In [42]:
df.head()

,title,overview,imdb_rating,tags
id,,,,
2,Ariel,a finnish man goes to the city to find a job a...,7.4,"[comedy, merjapulkkinen, romance, eetuhilkamo,..."
3,Shadows in Paradise,"nikander, a rubbish collector and would-be ent...",7.4,"[jukka-pekkapalo, romance, marirantasila, esko..."
5,Four Rooms,it's ted the bellhop's first night on the job....,6.7,"[robertrodriguez, quinnhellerman, quentintaran..."
6,Judgment Night,"four young friends, while taking a shortcut en...",6.6,"[lewiscolick, thriller, donovand.ross, nigelgi..."
8,Life in Loops (A Megacities RMX),timo novotny labels his new project an experim...,8.2,"[documentary, michaelglawogger, timonovotny]"


In [43]:
df=df[:30000]

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

# Ensure necessary NLTK data is downloaded
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Define custom tokenizer
def lemmatize_and_remove_stopwords(text):
    tokens = word_tokenize(text.lower())
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    filtered_tokens = [token for token in lemmatized_tokens if token not in stop_words]
    return filtered_tokens

# Initialize TfidfVectorizer with custom tokenizer
vectorizer = TfidfVectorizer(tokenizer=lemmatize_and_remove_stopwords, max_features=5000)

tfidf_matrix = vectorizer.fit_transform(df["overview"])

# To view the feature names
print(len(vectorizer.get_feature_names_out()))


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ethancoskay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ethancoskay/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ethancoskay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/Applications/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


5000


In [45]:
cosineSimilarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [57]:
print(cosineSimilarities.shape)
print(cosineSimilarities)

(30000, 30000)
[[1.         0.024673   0.04387235 ... 0.00383243 0.06050214 0.00559866]
 [0.024673   1.         0.08752498 ... 0.0090147  0.04043885 0.06671264]
 [0.04387235 0.08752498 1.         ... 0.0060158  0.00398379 0.03852898]
 ...
 [0.00383243 0.0090147  0.0060158  ... 1.         0.00366862 0.04857228]
 [0.06050214 0.04043885 0.00398379 ... 0.00366862 1.         0.05295783]
 [0.00559866 0.06671264 0.03852898 ... 0.04857228 0.05295783 1.        ]]


In [47]:
print(vectorizer.get_feature_names_out())

['!' '#' '$' ... '“' '”' '•']


In [48]:
def combined_similarity(idx, cos_sim_matrix, df):
    # Get the actual position of the given index
    idx_pos = df.index.get_loc(idx)

    results = []
    for i in df.index:
        i_pos = df.index.get_loc(i)

        # Calculate similarities
        jac = jaccard_similarity(df['tags'][idx], df['tags'][i])
        cos = cos_sim_matrix[idx_pos][i_pos]

        combined = (cos * 0.5 + jac * 0.5)
        results.append((i, combined))  # i is the actual index label
    
    return sorted(results, key=lambda x: x[1], reverse=True)[:11]


In [49]:
recommendations = combined_similarity(idx=680, cos_sim_matrix=cosineSimilarities, df=df)

In [50]:
recommendations

[(680, 1.0),
 (500, 0.1928285049681768),
 (9282, 0.1855852559583886),
 (20621, 0.1804616242594228),
 (28699, 0.17682940928179064),
 (40077, 0.16536520181578104),
 (45508, 0.1605416288838325),
 (37902, 0.15135328344716964),
 (35907, 0.14860371274300577),
 (9401, 0.14638256014746545),
 (9812, 0.14620839142565834)]

In [51]:
for pair in recommendations:
    print(df.loc[pair[0]].title)

Pulp Fiction
Reservoir Dogs
11:14
Second Chance
Underworld U.S.A.
Devil Face, Angel Heart
Return to a Better Tomorrow
Corrado
Kaante
2 Days in the Valley
Thursday


In [52]:
import pickle

with open('cosine_sim_matrix.pkl', 'wb') as f:
    pickle.dump(cosineSimilarities, f)

In [55]:
df.to_pickle('movies.pkl')